It is trivial to represnt a univariate polynomial on a computer simply as a list of coefficients for the terms. However creating a representation for a polynomial with multiple indeterminates is significantly more complicated. In particular we need to keep track of what indeterminates exist with what power in each and every term. Creating this directly would be a huge hassle.

A better way to handle this problem is to create a very restricted computer algebra system that allows us to build up the polynomials we want from simpler parts. I can't take credit for this idea which I got from a 2004 paper by Brent Dingle.

The first step is an Atom which just represents a single indeterminate raised to a power.

In [1]:
class Atom:    
    def __init__(self,s,p=1):
        assert type(s) == str
        assert len(s) == 1
        assert type(p) == int
        assert p >= 1
        
        self.s = s
        self.p = p

We will define operations with Atoms so that specifying *p* isn't necessary in general.

Next we will define a Particle which is a product of some Atoms with some coefficient that isn't an Atom. By not requiring that the coefficient be of any particular type we gain some forward compatibility if we have other numeric types. We also do not require that the Particle contain any Atoms at all which allows a Particle to be a constant.

In [2]:
class Particle:    
    def __init__(self,A,coef=1):
        assert type(A) == list
        assert all([type(a) == Atom for a in A])
        self.A = sorted(A)
        self.coef = coef

Note that the list of Atoms is sorted. We will see how this is done a bit later.

Finally we have the MVPoly class which consists of a sum of several particles represented as a list.

In [3]:
class MVPoly:
    def __init__(self,terms):
        self.terms = poly_merge(sorted(terms))


Again we will introduce what it means to sort and merge the terms of an MVPoly object later. 

To start with we want our classes to display themselves in a way that is easily comprehensible. This will make it much easier to check that things are working properly as we continue. For Atoms it is very simple. We just need to show the Atom being rasied to a power. The only special case is when the power is equal to 1 when we leave it out.

In [4]:
    def __str__(self):
        """Print the Atom"""
        if self.p == 1:
            return f"{self.s}"            
        return f"{self.s}^{self.p}"

For Particles, which could contain any number of Atoms, along with a coefficient things are singificantly more complicated with a lot of special cases. However having defined the way that Atoms should print simplified things a lot.

In [5]:
    def __str__(self):
        """Print the Particle"""
        if self.A == []:
            return str(self.coef)
        if self.coef == 1:
            out = ""
        elif self.coef == -1:
            out = "-"
        else:
            out = str(self.coef)
        for a in self.A:
            out += str(a)
        return out

Fortunately getting a polynomial to print correctly is fairly easy.

In [6]:
    def __str__(self):
        """Print the MVPoly"""
        out = str(self.terms[0])
        for term in self.terms[1:]:
            sgn = "-" if term.coef < 0 else "+"
            out +=  " " + sgn + " " + str(abs(term))
        return out

However getting a polynomial represented correctly involves more than just printing out the terms. We need to make sure that we have the correct terms and that they are ordered in way that makes some sense. Atoms we will just sort in terms of the symbol for their indeterminate. This is what is typical when actually writing polynomials.

In [7]:
    def __lt__(self,other):
        """For sorting atoms"""
        assert type(other) == Atom
        return self.s < other.s

As we saw above Particles use this when they are created. This ensure that identical Particles that have been given the Atoms in a different order still end up the same.

Ordering Particles is a bit complicated and arguably has no best method. Here we first sort to put Particles with many indeterminates to the left. For Particles with the same number of coefficients the ones with higher powers are sorted to the left.

In [8]:
    def __lt__(self,other):
        """For sorting Particles"""
        assert type(other) == Particle
        # First sort by number of indeterminates
        if len(self.A) != len(other.A):
            return len(self.A) > len(other.A)
        # If they are equal sort by the coefficients
        x = sum([a.p for a in self.A])
        y = sum([a.p for a in other.A])
        return x > y

Finally we need to have a way to deal with the possibility that a polynomial might end up with multiple identical terms or with terms with a zero coefficient. This can easily happen when adding or multiplying polynomials together.

Here are the functions needed.

In [9]:
from itertools import groupby

def particle_id(part):
    """Get the Particles indeterminates"""
    out = ""
    for a in part.A:
        out += str(a)
    return out

def poly_trim(L):
    """Remove terms with coef 0"""
    terms = []
    for t in L:
        if t.coef != 0:
            terms.append(t)
    return terms

def poly_merge(L):
    """Merge like terms"""
    terms = []
    for k,g in groupby(L,particle_id):
        G = list(g)
        t = G[0]
        for i in G[1:]:
            t += i
        terms.append(t)
    return poly_trim(terms)

This suffices to let us show polynomials correctly.

Now we need to create methods to combine Atom, Particles, and MVPoly objects in a useful way. This will be our very simple computer algebra system. It needs to accomodate addition, subtraction, multiplication, as well as exponentiation by nonnegative integers.

The primary complexity is that these operations do not have consistent results. For example *a + a* must produce a Particle object but *a + b* must produce an MVPoly object.

For Atoms here are the functions we will use.

In [10]:

    def __add__(self,other):
        """Addition of atoms with other objects"""
        return Particle([self]) + other


    def __radd__(self,other):
        """Addition of atoms with other objects"""
        return Particle([self]) + other


    def __sub__(self,other):
        """Subtraction other objects from atoms"""
        if type(other) == Atom:    
            return Particle([self]) - Particle([other])
        return Particle([self]) - other
    
    
    def __rsub__(self,other):
        """Subtraction of atoms from other objects"""
        return Particle([self],-1) + other


    def __neg__(self):
        return 0-self


    def __mul__(self,other):
        """Multiply atoms with other objects"""
        if type(other) == Atom:
            # Atoms with different indeterminate sum their powers
            if other.s == self.s:
                return Atom(self.s,self.p+other.p)
            else:
                # Atoms with different indeterminate form a particle
                return Particle([self,other])
        if type(other) == Particle:
            # When multiplied by a Particle use Particle multiplication
            return other*self
        # If it is something else make a particle with self as the 
        # indeterminate and other as the coefficient
        return Particle([self],other)


    def __rmul__(self,other):
        """Multiplication is commutative"""
        return self*other


    def __pow__(self,other):
        """Integer exponentiation"""
        assert type(other) == int
        assert other >= 0
        return Atom(self.s,self.p*other)

And then for Particles we need these for Particles

In [11]:
    def __mul__(self,other):
        # When multiplied by another particle merge their atoms then sort
        if type(other) == Particle:
            ownatoms = self.A.copy()
            atomtypes = [c.s for c in ownatoms]
            for a in other.A:
                if a.s not in atomtypes:
                    ownatoms.append(a)
                else:
                    for i in range(len(ownatoms)):
                        if ownatoms[i].s == a.s:
                            ownatoms[i] = ownatoms[i]*a
            ownatoms = sorted(ownatoms)
            return Particle(ownatoms,self.coef*other.coef)
        
        #  When multiplied by an atom merge the atom with it then sort
        if type(other) == Atom:
            ownatoms = self.A.copy()
            atomtypes = [at.s for at in ownatoms]
            if other.s not in atomtypes:
                ownatoms.append(other)
            else:
                for i in range(len(ownatoms)):
                    if ownatoms[i].s == other.s:
                        ownatoms[i] = ownatoms[i]*other
            ownatoms = sorted(ownatoms)
            return Particle(ownatoms,self.coef)
        
        # When multiplied by an number just change the coefficient
        return Particle(self.A,self.coef*other)


    def __rmul__(self,other):
        """Multiplication is commutative"""
        return self*other


    def __add__(self,other):
        if type(other) == Atom:
            return MVPoly([self,Particle([other])])
        if type(other) == Particle:
            if self.particle_id() == other.particle_id():
                return Particle(self.A,self.coef+other.coef)
            else:
                return MVPoly([self,other])
        return MVPoly([self,Particle([],other)])


    def __sub__(self,other):
        if type(other) == Atom:
            return MVPoly([self,Particle([other*-1])])
        if type(other) == Particle:
            return MVPoly([self,other*-1])
        return MVPoly([self,Particle([],-other)])


And finally for MVPoly

In [12]:
    def __mul__(self,other):
        """Multiply together two polynomials"""
        if type(other) == MVPoly:
            out = []
            for p in self.terms:
                for q in other.terms:
                    out.append(p*q)
            return MVPoly(out)
        else:
            out = []
            for p in self.terms:
                out.append(p*other)
            return MVPoly(out)

    
    def __add__(self,other):
        """Add together two polynomials"""
        if type(other) == MVPoly:
            return MVPoly(self.terms+other.terms)
        if type(other) == Particle:
            return MVPoly(self.terms+[other])
        if type(other) == Atom:
            return self + Particle([other])
        return self + Particle([],other)

Now we look at some example of how this works for us.

In [24]:
from Polynomials import Atom

a = Atom("a")
b = Atom("b")
c = Atom("c")

P1 = 5-a
P2 = a**3 + c + b
P3 = a

print(f"P1 = {P1}")
print(f"P2 = {P2}")
print(f"P1 * P2 = {P1*P2}")

P1 = -a + 5
P2 = a^3 + c + b
P1 * P2 = -ac - ab - a^4 + 5a^3 + 5c + 5b
